# Furniture Store Analysis

This notebook provides a comprehensive analysis of customer behavior and product performance in a furniture store. It includes:

- RFM (Recency, Frequency, Monetary) analysis for customer segmentation.
- Product performance evaluation by revenue and units sold.
- Customer review and rating trends.

---

## Importing Libraries

We start by importing the necessary Python libraries for data manipulation and analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

## Loading the Data

The datasets used include customer details, product information, orders, and customer reviews.

In [ ]:
customers = pd.read_csv('customers.csv')
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')
ratings = pd.read_csv('ratings.csv')

# Display the first few rows of each dataset
customers.head(), products.head(), orders.head(), ratings.head()

## RFM Analysis

RFM (Recency, Frequency, Monetary) analysis is performed to segment customers based on their purchase behavior.

In [ ]:
# Ensure 'order_date' is in datetime format
orders['order_date'] = pd.to_datetime(orders['order_date'])

# Calculate Recency, Frequency, and Monetary metrics
recency = orders.groupby('customer_id')['order_date'].max().reset_index()
recency['Recency'] = (datetime.now() - recency['order_date']).dt.days

frequency = orders.groupby('customer_id').size().reset_index(name='Frequency')

orders = orders.merge(products[['product_id', 'price']], on='product_id', how='left')
orders['revenue'] = orders['price'] * orders['quantity']
monetary = orders.groupby('customer_id')['revenue'].sum().reset_index(name='Monetary')

# Merge RFM metrics
rfm = recency[['customer_id', 'Recency']].merge(frequency, on='customer_id').merge(monetary, on='customer_id')
rfm.head()

## Product Performance Analysis

This section evaluates product performance based on revenue and units sold.

In [ ]:
# Calculate revenue and units sold per product
product_performance = orders.groupby('product_id').agg(
    total_revenue=('revenue', 'sum'),
    total_units_sold=('quantity', 'sum')
).reset_index()

# Merge with product details
product_performance = product_performance.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# Display top-performing products
product_performance.sort_values(by='total_revenue', ascending=False).head()

## Customer Reviews Analysis

Analyzing customer reviews to identify trends in satisfaction and product feedback.

In [ ]:
# Calculate the number of reviews and average rating per product
reviews_summary = ratings.groupby('product_id').agg(
    num_reviews=('rating', 'size'),
    avg_rating=('rating', 'mean')
).reset_index()

# Merge with product details
reviews_summary = reviews_summary.merge(products[['product_id', 'product_name']], on='product_id', how='left')

# Display top-reviewed products
reviews_summary.sort_values(by='num_reviews', ascending=False).head()

## Conclusion

This analysis provides valuable insights into customer behavior and product performance. The results can help optimize marketing strategies, improve inventory management, and enhance customer satisfaction.